
This Python script fetches current and historical prices for a list of futures tickers using the `yfinance` library, calculates percentage changes over different periods, and displays this information in a DataFrame. It includes two main functions:

### Functions

1. **`fetch_current_and_historical_prices(tickers)`:**
   - **Description:** Retrieves the last six months of price data for specified tickers.
   - **Returns:** The most recent closing price, as well as closing prices from the previous day, one month ago, three months ago, and six months ago.

2. **`display_data(run_continuously)`:**
   - **Description:** Repeatedly calls `fetch_current_and_historical_prices` to update and display price changes in real-time or for a single instance based on the `run_continuously` flag.
   - **Parameters:**
     - `run_continuously`: If set to `True`, it refreshes the data every 30 seconds; if `False`, it fetches the data just once.

### How to Use

- **Input:** Specify the list of tickers for the futures you want to monitor in the `futures_tickers` list.
- **Output:** The script displays a DataFrame showing the last price of each ticker and the percentage change from the previous day, one month ago, three months ago, and six months ago.
- **Execution:** Call `display_data(run_continuously=True)` to continuously monitor the prices or `display_data(run_continuously=False)` for a single snapshot of the data.
"""



In [ ]:
import yfinance as yf
import pandas as pd
import time
from datetime import datetime, timedelta

# List of futures tickers you want to monitor
futures_tickers = ['CL=F', 'GC=F', 'SI=F', 'ES=F', 'NQ=F']

def fetch_current_and_historical_prices(tickers):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=183)  # Fetch data for the last 6 months

    data = yf.download(tickers, start=start_date, end=end_date, progress=False)

    # Organize data into a DataFrame for easier management
    if len(tickers) > 1:
        last_price = data['Close'].iloc[-1]
        yesterday_price = data['Close'].shift(1).iloc[-1]
        one_month_ago_price = data['Close'].shift(30).iloc[-1]
        three_months_ago_price = data['Close'].shift(90).iloc[-1]
        six_months_ago_price = data['Close'].shift(180).iloc[-1]
    else:
        last_price = data.iloc[-1]['Close']
        yesterday_price = data.shift(1).iloc[-1]['Close']
        one_month_ago_price = data.shift(30).iloc[-1]['Close']
        three_months_ago_price = data.shift(90).iloc[-1]['Close']
        six_months_ago_price = data.shift(180).iloc[-1]['Close']

    return last_price, yesterday_price, one_month_ago_price, three_months_ago_price, six_months_ago_price

def calculate_percentage_change(current, historical):
    return ((current - historical) / historical) * 100

def display_data(run_continuously):
    while True:
        try:
            prices = fetch_current_and_historical_prices(futures_tickers)
            changes = {
                'Ticker': futures_tickers,
                'Last Price': prices[0],
                'Change from Yesterday (%)': calculate_percentage_change(prices[0], prices[1]),
                'Change from 1 Month Ago (%)': calculate_percentage_change(prices[0], prices[2]),
                'Change from 3 Months Ago (%)': calculate_percentage_change(prices[0], prices[3]),
                'Change from 6 Months Ago (%)': calculate_percentage_change(prices[0], prices[4])
            }
            df = pd.DataFrame(changes)
            display(df)
            if not run_continuously:
                break
            time.sleep(30)  # Refresh every 30 seconds
        except KeyboardInterrupt:
            print("Stopped by user.")
            break
        except Exception as e:
            print("Error fetching data:", e)
            time.sleep(30)  # Attempt to retry after 30 seconds

# Set run_continuously to True for continuous updates, or False for a single fetch
display_data(run_continuously=False)